In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [12]:
%%time
training_data_df = pd.read_csv('./Prepped Data/training_data_20220125.csv')
print(training_data_df.shape)
training_data_df.head()

(5068169, 19)
Wall time: 7.33 s


,주소_건축년도,전용면적(㎡),월세(만원),x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,거래유형,층,건축년도,부동산유형,브랜드,대장구분명,지목,구,동,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1991,25.35,12,2.066072,4.435861,4070,26,0.811539,0.584298,0,0,0,0,0,0,0,0,0,2253
1,서울특별시 강남구 개포동 12 1991,25.35,10,2.066072,4.435861,4133,26,0.895839,-0.444378,0,1,0,0,0,0,0,0,0,730
2,서울특별시 강남구 개포동 12 1991,25.35,4,2.066072,4.435861,4260,26,-0.880012,-0.474951,0,1,0,0,0,0,0,0,0,1815
3,서울특별시 강남구 개포동 12 1991,25.35,17,2.066072,4.435861,4470,27,0.999769,0.021516,0,2,0,0,0,0,0,0,0,3865
4,서울특별시 강남구 개포동 12 1991,25.35,7,2.066072,4.435861,4577,27,-0.247022,-0.969010,0,3,0,0,0,0,0,0,0,3062


In [2]:
%%time
coor_df = pd.read_csv('./Prepped Data/연속지적도출처_지번별좌표_baseline_20211222.csv')
print(coor_df.shape)
coor_df.head()

(922318, 7)
Wall time: 893 ms


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표
0,강남구,논현동,67-26 도,67-26,도,202648.630547,446277.538516
1,강남구,논현동,67-27 대,67-27,대,202676.093470,446297.563160
2,강남구,논현동,67-28 대,67-28,대,202584.375000,446275.930000
3,강남구,논현동,68 대,68,대,202695.885000,446317.525000
4,강남구,논현동,68-1 대,68-1,대,202707.844582,446324.121631


In [3]:
target_df = coor_df[
    (coor_df['시군구명'] == '금천구')
    &(coor_df['법정동명'] == '독산동')
    &(coor_df['지번'] == '181-31')
]
print(target_df.shape)
target_df.head()

(1, 7)


,시군구명,법정동명,지번_지목,지번,지목,x좌표,y좌표
279134,금천구,독산동,181-31대,181-31,대,191571.026664,440945.437093


In [5]:
import datetime

In [8]:
datetime.datetime.now()

datetime.datetime(2022, 1, 26, 12, 40, 32, 271252)

In [11]:
(pd.to_datetime(datetime.datetime.now()) - pd.to_datetime('2006-01-01')).days

5869

In [13]:
test_df = training_data_df.head(0)
print(test_df.shape)
test_df.head()

(0, 19)


,주소_건축년도,전용면적(㎡),월세(만원),x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,거래유형,층,건축년도,부동산유형,브랜드,대장구분명,지목,구,동,거래/보증금(만원)


In [16]:
def floor_feature_engi(x):
    # x is a number
    
    if x < 0:
        return '지하'
    elif x > 40:
        return '40층초과'
    else:
        return str(x)

In [17]:
def year_circular_sine_func(x):
    return np.sin(2*np.pi*((x)/365))

def year_circular_cosine_func(x):
    return np.cos(2*np.pi*((x)/365))

In [18]:
days_delta = (pd.to_datetime(datetime.datetime.now()) - pd.to_datetime('2006-01-01')).days
days_delta

5869

In [19]:
cat_col_maps_basedir = './Training/Training3_20220125/cat_col_maps/'

In [21]:
filenames = [f for f in os.listdir(cat_col_maps_basedir)]
filenames

['거래유형.xlsx',
 '건축년도.xlsx',
 '구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '지목.xlsx',
 '층.xlsx']

In [22]:
cat_cols = [col.replace('.xlsx', '') for col in filenames]
cat_cols

['거래유형', '건축년도', '구', '대장구분명', '동', '부동산유형', '브랜드', '지목', '층']

In [24]:
for file in filenames:
    print(file)
    map_df = pd.read_excel(cat_col_maps_basedir + file)
    print(map_df.shape)
    display(map_df)

거래유형.xlsx
(1, 3)


,월세,매매,전세
0,0,1,2


건축년도.xlsx
(1, 68)


,1991,1992,2004,1997,1982,2003,2011,1987,2002,1993,1995,2012,1988,2017,2001,2018,1990,2019,1994,1986,1983,1984,2010,2016,2013,2000,2015,1996,1989,2020,2021,1999,2014,1998,2007,2006,2005,2008,1976,1980,1978,2009,1979,1985,1981,1977,1974,1971,1973,1970,1975,1963,1972,1968,1969,1965,1962,1967,1961,1964,1966,1959,1960,1940,1956,1955,1925,1942
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67


구.xlsx
(1, 25)


...

,기타,현대,삼성,한신,푸르지오,래미안,아이파크,편한세상,아크로,아펠바움,롯데캐슬,더샵,코오롱,SK,힐스테이트,레미안,LG,데시앙,써밋,디에이치,리슈,대방,호반,포레나,중흥
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24


지목.xlsx
(1, 17)


,대,도,전,잡,공,답,임,차,장,철,주,학,종,수,사,창,과
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16


층.xlsx
(1, 43)


,12,8,11,9,1,5,4,6,14,7,15,10,3,13,2,16,17,21,22,19,18,20,25,26,23,24,지하,0,28,32,29,34,30,27,31,33,35,40층초과,37,36,40,38,39
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42


In [25]:
test_df.loc[0, '주소_건축년도'] = '서울특별시 금천구 독산동 181-31 2022'
test_df.loc[0, '전용면적(㎡)'] = 30
test_df.loc[0, '월세(만원)'] = 0
test_df.loc[0, 'x좌표'] = target_df['x좌표'].iloc[0]
test_df.loc[0, 'y좌표'] = target_df['y좌표'].iloc[0]
test_df.loc[0, '계약날짜일수_2006년기준'] = days_delta
test_df.loc[0, '계약날짜기준_건물연식'] = 0
test_df.loc[0, '계약날짜일수_sin'] = year_circular_sine_func(days_delta)
test_df.loc[0, '계약날짜일수_cos'] = year_circular_cosine_func(days_delta)
test_df.loc[0, '거래유형'] = '매매'
test_df.loc[0, '층'] = floor_feature_engi(1)
test_df.loc[0, '건축년도'] = 2022
test_df.loc[0, '부동산유형'] = '연립다세대'
test_df.loc[0, '브랜드'] = '기타'
test_df.loc[0, '대장구분명'] = '일반'
test_df.loc[0, '지목'] = '대'
test_df.loc[0, '구'] = '금천구'
test_df.loc[0, '동'] = '독산동'

# Training - Catboost Regressor

In [6]:
import catboost

In [7]:
from sklearn.model_selection import GroupKFold

In [8]:
gkfold = GroupKFold(n_splits=5)

In [10]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='RMSE')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_RMSE_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
Learning rate set to 0.079026
0:	learn: 29495.4096515	test: 29224.0890147	best: 29224.0890147 (0)	total: 343ms	remaining: 57m 8s
50:	learn: 13036.2735326	test: 13651.4947992	best: 13651.4947992 (50)	total: 8.79s	remaining: 28m 35s
100:	learn: 11379.1654507	test: 12334.9437856	best: 12334.9437856 (100)	total: 17.2s	remaining: 28m 4s
150:	learn: 10542.5587747	test: 11721.4955427	best: 11721.4955427 (150)	total: 25.7s	remaining: 27m 59s
200:	learn: 10031.5513174	test: 11435.2626320	best: 11435.2626320 (200)	total: 33.9s	remaining: 27m 33s
250:	learn: 9669.6783292	test: 11283.9674307	best: 11283.9674307 (250)	total: 42.2s	remaining: 27m 19s
300:	learn: 9382.0827457	test: 11219.8967093	best: 11213.1666885 (288)	total: 50.5s	remaining: 27m 6s
350:	learn: 9158.8962597	test: 11124.3210198	best: 11124.3210198 (350)	total: 58.8s	remaining: 26m 55s
400:	learn: 8963.6265008	test: 11074.1066151	best: 11074.1066151 (400)	total: 1m 7s	remaining: 26m 44s


250:	learn: 9861.4219674	test: 10049.9054928	best: 10049.9054928 (250)	total: 41.8s	remaining: 27m 2s
300:	learn: 9564.1568541	test: 9925.2213930	best: 9923.4452112 (299)	total: 49.8s	remaining: 26m 45s
350:	learn: 9315.2493622	test: 9873.6959672	best: 9873.1829971 (349)	total: 58.1s	remaining: 26m 38s
400:	learn: 9098.7784155	test: 9855.7160490	best: 9815.3642454 (394)	total: 1m 6s	remaining: 26m 31s
450:	learn: 8915.0367521	test: 9832.4736270	best: 9815.3642454 (394)	total: 1m 14s	remaining: 26m 24s
500:	learn: 8774.3403299	test: 9812.6628557	best: 9805.0522128 (495)	total: 1m 23s	remaining: 26m 14s
550:	learn: 8642.5615020	test: 9767.1231434	best: 9767.1231434 (550)	total: 1m 31s	remaining: 26m 6s
600:	learn: 8506.3816084	test: 9829.3290331	best: 9767.1231434 (550)	total: 1m 39s	remaining: 25m 59s
650:	learn: 8398.8237722	test: 9850.2720697	best: 9767.1231434 (550)	total: 1m 48s	remaining: 25m 53s
700:	learn: 8304.5587323	test: 9837.3226052	best: 9767.1231434 (550)	total: 1m 56s	rem

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.5')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_quantile05_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 9218.8632354	test: 9195.8661844	best: 9195.8661844 (0)	total: 380ms	remaining: 1h 3m 15s
50:	learn: 5179.7632508	test: 5196.1928033	best: 5196.1928033 (50)	total: 9.66s	remaining: 31m 24s
100:	learn: 4237.8242279	test: 4272.6669074	best: 4272.6669074 (100)	total: 19.5s	remaining: 31m 47s
150:	learn: 3796.8081943	test: 3829.9146556	best: 3829.9146556 (150)	total: 29.4s	remaining: 31m 57s
200:	learn: 3548.4239786	test: 3599.2751309	best: 3599.2751309 (200)	total: 39.5s	remaining: 32m 3s
250:	learn: 3407.7577231	test: 3473.5675875	best: 3473.5675875 (250)	total: 49.5s	remaining: 32m 1s
300:	learn: 3283.0732560	test: 3368.8800810	best: 3368.8800810 (300)	total: 59.7s	remaining: 32m 4s
350:	learn: 3167.7960210	test: 3271.9725818	best: 3271.9725818 (350)	total: 1m 9s	remaining: 31m 49s
400:	learn: 3075.8375991	test: 3201.8166399	best: 3201.8166399 (400)	total: 1m 19s	remaining: 31m 42s
450:	learn: 2987.5207924	test: 3135.5768413	best: 

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.25')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_quantile025_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.75')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_quantile075_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.125')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_quantile0125_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.875')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training3_20220125/catboost_quantile0875_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1